# Cleaning The Raw Data

## Import Depnedencies

In [1]:
import pandas as pd
import numpy as np
import os, sys

## Columns, Filenames, and Directories
Some variables which describe the column names/types, filenames and input/output directories 

In [2]:
# column names for datafiles
headers=['id', 'pricing', 'datetime',
         'KwH/hh', 'acorn', 'acorn-grouped']

# column types for datafile
dtypes={'KwH/hh': 'float'}


# format-string for datafile path
pathfmt=os.path.join('.', '{dir}','{year}-{month:02}-power-survey-london.csv{suffix}')

# directories
input_dir=os.path.join('..', 'raw')
output_dir=os.path.join('..', 'data')


These next variables select the year and month we are going to load and clean; since these files are ~500MB uncompressed, we only load them one at a time. *(The script which does all of this for every relavent year and month is located in the "scripts" directory of the repository.)*

In [3]:
# year to load
year = 2012
# month (1 to 12) of the year to load 
month = 7

## Create Output Directory (if it doesn't exist)

In [4]:
try: # try to make output directory
    os.mkdir(output_dir)
except FileExistsError: # if it exists, do nothing.
    pass

## Generate Input/Output Filenames


In [5]:
output_path = pathfmt.format( year=year, month=month, dir=output_dir, suffix='' )
input_path = pathfmt.format( year=year, month=month, dir=input_dir, suffix='.bz2')

print(f"Processing file {input_path}")

Processing file .\..\raw\2012-07-power-survey-london.csv.bz2


## Load Raw Data

In [6]:
data = pd.read_csv( input_path,    # file to load 
                    header=None,   # month/year files do not have a header 
                    names=headers )# names to use for the columns 

### Extract Non-Nil Records 
Only look at records with valid power usage reading; entries without valid readings have 'Null' where the power usage should be. In addition, make sure cleaned 'KwH/hh' column consists of floats.

In [7]:
tmp = data.copy()
tmp = tmp.loc[ tmp['KwH/hh'] != 'Null' ]
data = tmp.astype( dtypes ).copy()
del tmp # free up some resources

C:\Users\burned\anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


### Fix Datetime And ACORN Column

In [8]:
# Note: this next bit only works because the time format is close to 'normal'
data['datetime'] = pd.to_datetime( data['datetime'] )

# Remove "ACORN-" from each entry in the 'acorn' column; keeping only the group letter
data['acorn'] = data['acorn'].apply( lambda x : x[-1] )

# take care of possible double counting
data = data.groupby(['id', 'datetime', 'acorn', 'acorn-grouped']).mean().reset_index()

### Compute Values For Each ACORN Groups

In [9]:
# group data by datetime and acorn type
grouped = data.groupby(['datetime', 'acorn'])

# compute/combine values for each datetime and  acorn type 
sums = grouped.sum() # combine the power usage
stds = grouped.std() # compute sample standard deviation
counts = grouped.count() # count number of records for each datetime and acorn type

# merge all three values into one table
merged = pd.merge(counts['id'], sums, 
                  left_index=True, right_index=True)
merged = pd.merge(merged, stds, 
                  left_index=True, right_index=True)
merged = merged.reset_index()

### Mung Merged Data
Create columns which correspond to the ACORN type associated with the values we computed then merge results.

In [10]:
# group by datetime
grouped = merged.groupby('datetime')

# extract and mung computed values
counts_atyp = grouped.apply( lambda x : \
                          x.set_index('acorn').transpose().iloc[1])
sums_atyp = grouped.apply( lambda x : \
                         x.set_index('acorn').transpose().iloc[2])
stds_atyp = grouped.apply(lambda x : \
                         x.set_index('acorn').transpose().iloc[3])

# fixup columns names
counts_atyp = counts_atyp.rename(
    index=str,
    columns=dict([(name, name+'_count') for name in counts_atyp.columns]))
sums_atyp = sums_atyp.rename(
    index=str,
    columns=dict([(name, name+'_sigma') for name in sums_atyp.columns]))
stds_atyp = stds_atyp.rename(
    index=str,
    columns=dict([(name, name+'_std') for name in stds_atyp.columns]))

# merge everything into one dataframe
combined = pd.merge(sums_atyp,  stds_atyp,
                   left_index=True, right_index=True)
combined = pd.merge(combined, counts_atyp,
                   left_index=True, right_index=True)

### Add Summary Columns

In [18]:
count_total = counts_atyp.sum(axis=1)
sigma_total = sums_atyp.sum(axis=1)
means = sigma_total / count_total

combined['sigma'] = sigma_total
combined['count'] = count_total
combined['std'] = data.groupby('datetime').std()
combined['mean'] = means

### Preview Combined Data

In [19]:
# preview a sample from the combined data
combined.sample(5)

acorn,A_sigma,B_sigma,C_sigma,D_sigma,E_sigma,F_sigma,G_sigma,H_sigma,I_sigma,J_sigma,...,N_count,O_count,P_count,Q_count,U_count,sigma,count,mean,stds,std
datetime,,,,,,,,,,,,,,,,,,,,,
2012-07-31 07:30:00,29.020,2.245,17.247,50.806,211.524,79.313,17.783,39.523,4.926,7.159,...,88,61,83,675,30,649.112000,3785,0.171496,0.217204,0.217204
2012-07-21 20:30:00,34.115,2.432,19.061,61.806,268.188,113.531,21.912,60.206,7.200,11.631,...,87,61,82,673,29,866.749000,3781,0.229238,0.247175,0.247175
2012-07-12 06:30:00,33.863,1.374,13.944,48.103,206.964,84.519,17.481,45.824,5.945,10.560,...,87,60,83,662,27,652.901000,3701,0.176412,0.255216,0.255216
2012-07-06 08:00:00,22.013,2.008,13.561,50.855,211.340,86.431,16.429,44.742,3.784,7.559,...,84,56,78,622,23,640.615999,3478,0.184191,0.224658,0.224658
2012-07-08 07:30:00,25.032,1.341,12.559,42.299,175.619,75.695,20.244,40.940,3.110,7.810,...,86,58,80,638,26,574.659000,3580,0.160519,0.205303,0.205303


## Save Combined Data

In [20]:
print(f"Outputing to file {output_path}")

combined.to_csv(output_path)


Outputing to file .\..\data\2012-07-power-survey-london.csv
